In [1]:
import numpy as np
from pymongo import MongoClient
import sys
sys.path.append('../statistical_calculations')  # Adds the statistical_calculations directory to the path
import disproportionaly_analysis
from tqdm import tqdm

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client['vaers']  
collection = db['combinations COVID']

### Adverse Reactions

In [3]:
Pfizer_adverse_reactions = [
    "Vomiting",
    "Lymphadenopathy",
    "Rash", 
    "Pruritus", 
    "Urticaria", 
    "Angioedema",
    "Anaphylaxis",
    "Decreased appetite",
    "Insomnia",
    "Headache",
    "Dizziness",
    "Lethargy",
    #"Acute peripheral facial paralysis",
    "Facial paralysis",
    "Paraesthesia",
    "Hypoaesthesia",
    "Myocarditis",
    "Pericarditis",
    "Diarrhoea",
    "Nausea",
    "Hyperhidrosis",
    "Night sweats",
    "Erythema multiforme",
    "Arthralgia",
    "Myalgia",
    "Pain in extremity",
    "Heavy menstrual bleeding",
    "Injection site pain",
    "Fatigue",
    "Chills",
    "Pyrexia",
    "Injection site swelling",
    "Injection site redness",
    "Asthenia",
    "Malaise",
    "Injection site pruritus",
    "Extensive swelling of vaccinated limb",
    "Facial swelling"
]

In [4]:
Moderna_adverse_reactions = [
    "Lymphadenopathy",
    "Anaphylaxis",
    "Hypersensitivity",
    "Decreased appetite",
    "Irritability", 
    "Crying",
    "Headache",
    "Sleepiness",
    "Dizziness",
    #"Acute peripheral facial paralysis",
    "Facial paralysis", #instead of facial paralysis
    "Hypoaesthesia",
    "Paraesthesia",
    "Myocarditis",
    "Pericarditis",
    "Nausea", 
    "Vomiting",
    "Diarrhoea",
    "Abdominal pain",
    "Rash",
    "Urticaria",
    "Erythema multiforme",
    "Mechanical urticaria",
    "Chronic urticaria",
    "Myalgia",
    "Arthralgia",
    "Heavy menstrual bleeding",
    "Injection site pain",
    "Fatigue",
    "Chills",
    "Pyrexia",
    "Injection site swelling",
    "Injection site erythema",
    "Injection site urticaria",
    "Injection site rash",
    "Delayed injection site reaction",
    "Injection site pruritus",
    "Facial swelling",
    "Extensive swelling of vaccinated limb"
]


In [5]:
Janssen_adverse_reactions = [
    "Lymphadenopathy",
    "Immune thrombocytopenia",
    "Anaphylaxis",
    "Headache",
    "Dizziness",
    "Tremor",
    "Urticaria",
    "Hypersensitivity",
    "Paraesthesia",
    "Hypoaesthesia",
    "Facial paralysis",
    "Tinnitus",
    "Guillain-Barre syndrome",
    #"Transverse myelitis", not in DB
    "Myelitis", #instead of Transverse myelitis
    "Venous thromboembolism",
    "Thrombosis in combination with thrombocytopenia",
    "Myocarditis", 
    "Pericarditis",
    "Capillary leak syndrome",
    #"Small vessel cutaneous vasculitis", not in DB
    "Cutaneous vasculitis", #instead of Small vessel cutaneous vasculitis"
    "Nausea",
    "Cough", 
    "Oropharyngeal pain",
    "Sneezing",
    "Diarrhoea",
    "Vomiting",
    "Rash",
    "Hyperhidrosis",
    "Myalgia",
    "Arthralgia",
    "Muscular weakness",
    "Back pain",
    "Pain in extremity",
    "Fatigue",
    "Injection site pain",
    "Injection site swelling",
    "Chills",
    "Pyrexia"
]

In [6]:
Novavax_adverse_reactions = [
    "Headache",
    "Nausea",
    "Vomiting",
    "Myalgia",
    "Arthralgia",
    "Injection site tenderness",
    "Injection site pain",
    "Fatigue",
    "Malaise",
    "Injection site redness",
    "Injection site swelling",
    "Pyrexia",
    "Pain in extremity",
    "Injection site pruritus",
    "Chills",
    "Injection site warmth",
    "Lymphadenopathy",
    "Anaphylaxis",
    "Paraesthesia",
    "Hypoaesthesia",
    "Myocarditis",
    "Pericarditis",
    "Hypertension",
    "Rash",
    "Erythema",
    "Pruritus",
    "Urticaria"
]

### PFIZER

In [17]:
query = {
      "vaccine": "COVID19",
      "manufacturer": "PFIZER\\BIONTECH"
}
y_true_Pfizer = []
rrr_chi2_Pfizer = []
rrr_ci_Pfizer = []
prr_chi2_Pfizer = []
prr_ci_Pfizer = []
ror_chi2_Pfizer = []
ror_ci_Pfizer = []
ic_chi2_Pfizer = []
ic_ci_Pfizer = []
order_Pfizer = []

results = list(collection.find(query))

for result in tqdm(results):
    DE = result.get("DE")
    if DE >= 3:
        dE = result.get("dE")
        De = result.get("De")
        de = result.get("de")

        # Ensure all necessary data is available
        if None in [DE, dE, De, de]:
            continue

        contingency_table = [
            [DE, dE],
            [De, de]
        ]

        # Compute statistical measures
        rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(
            contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
        rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)

        prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(
            contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
        prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)

        ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(
            contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
        ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)

        chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
        IC = disproportionaly_analysis.information_component(contingency_table)
        IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(
            contingency_table, IC)

        order_Pfizer.append(result["symptom"])

        if result.get("symptom") in Pfizer_adverse_reactions:
            y_true_Pfizer.append(1)
        else:
            y_true_Pfizer.append(0)

        # CHI SQUARE METHODS
        print(rrr)
        if rrr > 2 and chi_square > 3.841:
            rrr_chi2_Pfizer.append(1)
        else:
            rrr_chi2_Pfizer.append(0)

        if prr > 2 and chi_square > 3.841:
            prr_chi2_Pfizer.append(1)
        else:
            prr_chi2_Pfizer.append(0)

        if ror > 2 and chi_square > 3.841:
            ror_chi2_Pfizer.append(1)
        else:
            ror_chi2_Pfizer.append(0)

        if IC > 0 and chi_square > 3.841:  #bedingung?
            ic_chi2_Pfizer.append(1)
        else:
            ic_chi2_Pfizer.append(0)

        # CI METHODS

        if rrr_ci_lower > 1:
            rrr_ci_Pfizer.append(1)
        else:
            rrr_ci_Pfizer.append(0)

        if prr_ci_lower > 1:
            prr_ci_Pfizer.append(1)
        else:
            prr_ci_Pfizer.append(0)

        if ror_ci_lower > 1:
            ror_ci_Pfizer.append(1)
        else:
            ror_ci_Pfizer.append(0)

        if IC_ci_lower > 0:
            ic_ci_Pfizer.append(1)
        else:
            ic_ci_Pfizer.append(0)

    

100%|██████████| 9886/9886 [00:00<00:00, 73811.41it/s]

1.1429427794647047
0.8572070845985285
1.2858106268977927
1.4286784743308807
0.9916709410061407
1.3878590893499985
1.4694978593117631
1.0592240408414937
1.714414169197057
1.4027025020703192
1.0401073108753354
1.3220810100959426
1.028648501518234
0.7273272232957211
1.0204846245220578
1.180513562872714
1.333433242708822
1.3715313353576455
1.2924045275485507
0.9351350013802129
0.8171038291787143
1.186902117136424
1.040894317012499
1.2468466685069506
1.4172490465362337
1.500112398047425
1.183762164445587
1.170155702785293
1.714414169197057
0.9587495216170869
1.333433242708822
0.8231136210065415
1.3797825265462944
0.7347489296558816
1.0715088557481607
1.1626486894554755
1.273564811403528
1.120963110628845
1.714414169197057
0.7403152094260018
1.0909908349435817
1.4286784743308807
1.333433242708822
0.9231460911061076
1.2858106268977927
1.5239237059529396
1.714414169197057
1.714414169197057
1.714414169197057
1.5429727522773513
1.0361843879762431
1.4102439133717726
1.1429427794647047
1.359707789

In [8]:
np.array(rrr_chi2_Pfizer).dump(open('Covid_data/Arrays_Pfizer/rrr_chi2_Pfizer.npy', 'wb'))
np.array(prr_chi2_Pfizer).dump(open('Covid_data/Arrays_Pfizer/prr_chi2_Pfizer.npy', 'wb'))
np.array(ror_chi2_Pfizer).dump(open('Covid_data/Arrays_Pfizer/ror_chi2_Pfizer.npy', 'wb'))
np.array(ic_chi2_Pfizer).dump(open('Covid_data/Arrays_Pfizer/ic_chi2_Pfizer.npy', 'wb'))

np.array(rrr_ci_Pfizer).dump(open('Covid_data/Arrays_Pfizer/rrr_ci_Pfizer.npy', 'wb'))
np.array(prr_ci_Pfizer).dump(open('Covid_data/Arrays_Pfizer/prr_ci_Pfizer.npy', 'wb'))
np.array(ror_ci_Pfizer).dump(open('Covid_data/Arrays_Pfizer/ror_ci_Pfizer.npy', 'wb'))
np.array(ic_ci_Pfizer).dump(open('Covid_data/Arrays_Pfizer/ic_ci_Pfizer.npy', 'wb'))

np.array(y_true_Pfizer).dump(open('Covid_data/Arrays_Pfizer/y_true_Pfizer.npy', 'wb'))
np.array(order_Pfizer).dump(open('Covid_data/Arrays_Pfizer/order_Pfizer.npy', 'wb'))

### Moderna

In [9]:
query = {
      "vaccine": "COVID19",
      "manufacturer": "MODERNA"
}
y_true_Moderna = []
rrr_chi2_Moderna = []
rrr_ci_Moderna = []
prr_chi2_Moderna = []
prr_ci_Moderna = []
ror_chi2_Moderna = []
ror_ci_Moderna = []
ic_chi2_Moderna = []
ic_ci_Moderna = []
order_Moderna = []

results = list(collection.find(query))

for result in tqdm(results): 
    DE = result.get("DE")
    if DE >=3:
        dE = result.get("dE")
        De = result.get("De")
        de = result.get("de")
        
        # Ensure all necessary data is available
        if None in [DE, dE, De, de]:
            continue
        
        contingency_table = [
            [DE, dE],
            [De, de]
        ]
        
        # Compute statistical measures
        rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
        rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
        
        prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
        prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
        
        ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
        ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
        
        chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
        IC = disproportionaly_analysis.information_component(contingency_table)
        IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
        
        order_Moderna.append(result["symptom"])
        
        if result.get("symptom") in Moderna_adverse_reactions:
            y_true_Moderna.append(1)
        else:
            y_true_Moderna.append(0)
            
            
        # CHI SQUARE METHODS
            
        if rrr > 2 and chi_square > 3.841:
            rrr_chi2_Moderna.append(1)
        else:
            rrr_chi2_Moderna.append(0)
            
        if prr > 2 and chi_square > 3.841:
            prr_chi2_Moderna.append(1)
        else:
            prr_chi2_Moderna.append(0)
        
        if ror > 2 and chi_square > 3.841:
            ror_chi2_Moderna.append(1)
        else:
            ror_chi2_Moderna.append(0)
        
        if IC > 0 and chi_square > 3.841: #bedingung?
            ic_chi2_Moderna.append(1)
        else: 
            ic_chi2_Moderna.append(0)
            
        # CI METHODS
        
        if rrr_ci_lower > 1:
            rrr_ci_Moderna.append(1)
        else:
            rrr_ci_Moderna.append(0)
            
        if prr_ci_lower > 1:
            prr_ci_Moderna.append(1)
        else:
            prr_ci_Moderna.append(0)
        
        if ror_ci_lower > 1:
            ror_ci_Moderna.append(1)
        else:
            ror_ci_Moderna.append(0)
        
        if IC_ci_lower > 0:
            ic_ci_Moderna.append(1)
        else: 
            ic_ci_Moderna.append(0)

100%|██████████| 7217/7217 [00:00<00:00, 93014.29it/s]


In [10]:
np.array(rrr_chi2_Moderna).dump(open('Covid_data/Arrays_Moderna/rrr_chi2_Moderna.npy', 'wb'))
np.array(prr_chi2_Moderna).dump(open('Covid_data/Arrays_Moderna/prr_chi2_Moderna.npy', 'wb'))
np.array(ror_chi2_Moderna).dump(open('Covid_data/Arrays_Moderna/ror_chi2_Moderna.npy', 'wb'))
np.array(ic_chi2_Moderna).dump(open('Covid_data/Arrays_Moderna/ic_chi2_Moderna.npy', 'wb'))

np.array(rrr_ci_Moderna).dump(open('Covid_data/Arrays_Moderna/rrr_ci_Moderna.npy', 'wb'))
np.array(prr_ci_Moderna).dump(open('Covid_data/Arrays_Moderna/prr_ci_Moderna.npy', 'wb'))
np.array(ror_ci_Moderna).dump(open('Covid_data/Arrays_Moderna/ror_ci_Moderna.npy', 'wb'))
np.array(ic_ci_Moderna).dump(open('Covid_data/Arrays_Moderna/ic_ci_Moderna.npy', 'wb'))

np.array(y_true_Moderna).dump(open('Covid_data/Arrays_Moderna/y_true_Moderna.npy', 'wb'))
np.array(order_Moderna).dump(open('Covid_data/Arrays_Moderna/order_Moderna.npy', 'wb'))

In [11]:
print(len(rrr_chi2_Moderna))
print(len(prr_chi2_Moderna))
print(len(ror_chi2_Moderna))
print(len(ic_chi2_Moderna))
print(len(rrr_ci_Moderna))
print(len(prr_ci_Moderna))
print(len(ror_ci_Moderna))
print(len(ic_ci_Moderna))
print(len(order_Moderna))

6899
6899
6899
6899
6899
6899
6899
6899
6899


### Janssen

In [12]:
query = {
      "vaccine": "COVID19",
      "manufacturer": "JANSSEN"
}
y_true_Janssen = []
rrr_chi2_Janssen = []
rrr_ci_Janssen = []
prr_chi2_Janssen = []
prr_ci_Janssen = []
ror_chi2_Janssen = []
ror_ci_Janssen = []
ic_chi2_Janssen = []
ic_ci_Janssen = []
order_Janssen = []

results = list(collection.find(query))

for result in tqdm(results): 
    DE = result.get("DE")
    if DE >=3:
        dE = result.get("dE")
        De = result.get("De")
        de = result.get("de")
        
        # Ensure all necessary data is available
        if None in [DE, dE, De, de]:
            continue
        
        contingency_table = [
            [DE, dE],
            [De, de]
        ]
        
        # Compute statistical measures
        rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
        rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
        
        prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
        prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
        
        ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
        ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
        
        chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
        IC = disproportionaly_analysis.information_component(contingency_table)
        IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
        
        order_Janssen.append(result["symptom"])
        
        if result.get("symptom") in Janssen_adverse_reactions:
            y_true_Janssen.append(1)
        else:
            y_true_Janssen.append(0)
            
            
        # CHI SQUARE METHODS
            
        if rrr > 2 and chi_square > 3.841:
            rrr_chi2_Janssen.append(1)
        else:
            rrr_chi2_Janssen.append(0)
            
        if prr > 2 and chi_square > 3.841:
            prr_chi2_Janssen.append(1)
        else:
            prr_chi2_Janssen.append(0)
        
        if ror > 2 and chi_square > 3.841:
            ror_chi2_Janssen.append(1)
        else:
            ror_chi2_Janssen.append(0)
        
        if IC > 0 and chi_square > 3.841: #bedingung?
            ic_chi2_Janssen.append(1)
        else: 
            ic_chi2_Janssen.append(0)
            
        # CI METHODS
        
        if rrr_ci_lower > 1:
            rrr_ci_Janssen.append(1)
        else:
            rrr_ci_Janssen.append(0)
            
        if prr_ci_lower > 1:
            prr_ci_Janssen.append(1)
        else:
            prr_ci_Janssen.append(0)
        
        if ror_ci_lower > 1:
            ror_ci_Janssen.append(1)
        else:
            ror_ci_Janssen.append(0)
        
        if IC_ci_lower > 0:
            ic_ci_Janssen.append(1)
        else: 
            ic_ci_Janssen.append(0)
    

100%|██████████| 3674/3674 [00:00<00:00, 86784.89it/s]


In [13]:
np.array(rrr_chi2_Janssen).dump(open('Covid_data/Arrays_Janssen/rrr_chi2_Janssen.npy', 'wb'))
np.array(prr_chi2_Janssen).dump(open('Covid_data/Arrays_Janssen/prr_chi2_Janssen.npy', 'wb'))
np.array(ror_chi2_Janssen).dump(open('Covid_data/Arrays_Janssen/ror_chi2_Janssen.npy', 'wb'))
np.array(ic_chi2_Janssen).dump(open('Covid_data/Arrays_Janssen/ic_chi2_Janssen.npy', 'wb'))

np.array(rrr_ci_Janssen).dump(open('Covid_data/Arrays_Janssen/rrr_ci_Janssen.npy', 'wb'))
np.array(prr_ci_Janssen).dump(open('Covid_data/Arrays_Janssen/prr_ci_Janssen.npy', 'wb'))
np.array(ror_ci_Janssen).dump(open('Covid_data/Arrays_Janssen/ror_ci_Janssen.npy', 'wb'))
np.array(ic_ci_Janssen).dump(open('Covid_data/Arrays_Janssen/ic_ci_Janssen.npy', 'wb'))

np.array(y_true_Janssen).dump(open('Covid_data/Arrays_Janssen/y_true_Janssen.npy', 'wb'))
np.array(order_Janssen).dump(open('Covid_data/Arrays_Janssen/order_Janssen.npy', 'wb'))

### Novavax

In [14]:
query = {
      "vaccine": "COVID19",
      "manufacturer": "NOVAVAX"
}
y_true_Novavax = []
rrr_chi2_Novavax = []
rrr_ci_Novavax = []
prr_chi2_Novavax = []
prr_ci_Novavax = []
ror_chi2_Novavax = []
ror_ci_Novavax = []
ic_chi2_Novavax = []
ic_ci_Novavax = []
order_Novavax = []

results = list(collection.find(query))

for result in tqdm(results): 
    DE = result.get("DE")
    if DE >=3:
        dE = result.get("dE")
        De = result.get("De")
        de = result.get("de")
        
        # Ensure all necessary data is available
        if None in [DE, dE, De, de]:
            continue
        
        contingency_table = [
            [DE, dE],
            [De, de]
        ]
        
        # Compute statistical measures
        rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
        rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
        
        prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
        prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
        
        ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
        ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
        
        chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
        IC = disproportionaly_analysis.information_component(contingency_table)
        IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
        
        order_Novavax.append(result["symptom"])
        
        if result.get("symptom") in Novavax_adverse_reactions:
            y_true_Novavax.append(1)
        else:
            y_true_Novavax.append(0)
            
            
        # CHI SQUARE METHODS
            
        if rrr > 2 and chi_square > 3.841:
            rrr_chi2_Novavax.append(1)
        else:
            rrr_chi2_Novavax.append(0)
            
        if prr > 2 and chi_square > 3.841:
            prr_chi2_Novavax.append(1)
        else:
            prr_chi2_Novavax.append(0)
        
        if ror > 2 and chi_square > 3.841:
            ror_chi2_Novavax.append(1)
        else:
            ror_chi2_Novavax.append(0)
        
        if IC > 0 and chi_square > 3.841: #bedingung?
            ic_chi2_Novavax.append(1)
        else: 
            ic_chi2_Novavax.append(0)
            
        # CI METHODS
        
        if rrr_ci_lower > 1:
            rrr_ci_Novavax.append(1)
        else:
            rrr_ci_Novavax.append(0)
            
        if prr_ci_lower > 1:
            prr_ci_Novavax.append(1)
        else:
            prr_ci_Novavax.append(0)
        
        if ror_ci_lower > 1:
            ror_ci_Novavax.append(1)
        else:
            ror_ci_Novavax.append(0)
        
        if IC_ci_lower > 0:
            ic_ci_Novavax.append(1)
        else: 
            ic_ci_Novavax.append(0)
        


100%|██████████| 159/159 [00:00<00:00, 39330.88it/s]


In [15]:
np.array(rrr_chi2_Novavax).dump(open('Covid_data/Arrays_Novavax/rrr_chi2_Novavax.npy', 'wb'))
np.array(prr_chi2_Novavax).dump(open('Covid_data/Arrays_Novavax/prr_chi2_Novavax.npy', 'wb'))
np.array(ror_chi2_Novavax).dump(open('Covid_data/Arrays_Novavax/ror_chi2_Novavax.npy', 'wb'))
np.array(ic_chi2_Novavax).dump(open('Covid_data/Arrays_Novavax/ic_chi2_Novavax.npy', 'wb'))

np.array(rrr_ci_Novavax).dump(open('Covid_data/Arrays_Novavax/rrr_ci_Novavax.npy', 'wb'))
np.array(prr_ci_Novavax).dump(open('Covid_data/Arrays_Novavax/prr_ci_Novavax.npy', 'wb'))
np.array(ror_ci_Novavax).dump(open('Covid_data/Arrays_Novavax/ror_ci_Novavax.npy', 'wb'))
np.array(ic_ci_Novavax).dump(open('Covid_data/Arrays_Novavax/ic_ci_Novavax.npy', 'wb'))

np.array(y_true_Novavax).dump(open('Covid_data/Arrays_Novavax/y_true_Novavax.npy', 'wb'))
np.array(order_Novavax).dump(open('Covid_data/Arrays_Novavax/order_Novavax.npy', 'wb'))

In [16]:
print(len(rrr_chi2_Novavax))
print(len(prr_chi2_Novavax))
print(len(ror_chi2_Novavax))
print(len(ic_chi2_Novavax))
print(len(rrr_ci_Novavax))
print(len(prr_ci_Novavax))
print(len(ror_ci_Novavax))
print(len(ic_ci_Novavax))
print(len(order_Novavax))

151
151
151
151
151
151
151
151
151
